In [1]:
import numpy as np
import pandas as pd
import sympy as sp

from astropy import units as u
from astropy import constants as const
from astropy.units import imperial
imperial.enable()

In [2]:
# Critical Inputs

V_star = 0.1

H_star = 7.0


print(f"1/H* = {1/H_star}")
if V_star <= 1/H_star:
  print("Will accumulate")
else:
  print("Will not accumulate")

1/H* = 0.14285714285714285
Will accumulate


In [3]:
# Hardcode Inputs

D_f = 1.75 # Diameter of filament in 
D_n = 0.4 # Diameter of Nozzle

T_n = 215 # Nozzle Temp (°C)
T_b = 60 # Bed Temp (°C)

X0 = 50 # Starting move x-coordinate (mm)
Y0 = 105 # Starting move y-coordinate (mm)

Z0 = 5 # Height of first layer, [AKA: H] (mm)

F1 = 300 # Second coordinates F-speed, [move speed of nozzle to second coordinates] (mm/min)
# X1 = 200 # Second move x-coordinate (mm)
# E1 = 100 # Second move E-amount, [amount of filament fed into the extruder during move to second coordinates] (mm)
S = 1 # Flow Multiplyer, [M221]

L1 = 150 # Length of move (mm)

α = 1.63 # Die-Swell Constant [Prediction]
a = α

In [4]:
# Formulaic Inputs

L = L1

H = H_star*α*D_n

E = (L/(S*V_star))*(((α*D_n)/D_f)**2)

Z0 = H
E1 = E

X1 = X0 + L1

print(f"H = {H}")
print(f"E = {E}")
print(f"X1 = {X1}")
print(f"")

H = 4.564
E = 208.2142040816327
X1 = 200



In [5]:
file1 = open("Sample.gcode","w") 

D1 = [f'M73 P0 R0 \n',
      f'M201 X9000 Y9000 Z500 E10000 ; sets maximum accelerations, mm/sec^2 \n',
      f'M203 X500 Y500 Z12 E120 ; sets maximum feedrates, mm/sec \n',
      f'M204 P2000 R1500 T2000 ; sets acceleration (P, T) and retract acceleration (R), mm/sec^2 \n',
      f'M205 X10.00 Y10.00 Z0.20 E4.50 ; sets the jerk limits, mm/sec \n',
      f'M205 S0 T0 ; sets the minimum extruding and travel feed rate, mm/sec \n',
      f'M107 \n',
      f'M862.3 P "MK2.5S" ; printer model check \n',
      f'M862.1 P0.4 ; nozzle diameter check \n',
      f'M115 U3.9.1 ; tell printer latest fw version \n',
      f'G90 ; use absolute coordinates \n',
      f'M83  ; extruder relative mode \n',
      
      f'\n'
      
      f'M104 S{T_n} ; set extruder temp \n',
      f'M140 S{T_b} ; set bed temp \n',
      
      f'\n',
      
      f'M190 S60 ; wait for bed temp \n',
      f'M109 S215 ; wait for extruder temp \n',
      f'G28 W ; home all without mesh bed level \n',
      f'G80 ; mesh bed leveling \n',
      f'G1 Y-3.0 F1000.0 ; go outside print area \n',
      f'G92 E0.0 \n',
      f'G1 X60.0 E9.0 F1000.0 ; intro line \n',
      f'M73 P14 R0 \n',
      f'G1 X100.0 E12.5 F1000.0 ; intro line \n',
      f'G92 E0.0 \n',
      f'G21 ; set units to millimeters \n',
      f'G90 ; use absolute coordinates \n',
      f'M83 ; use relative distances for extrusion \n',
      f'M900 K0.05 ; Filament gcode LA 1.5 \n',
      f'M900 K30 ; Filament gcode LA 1.0 \n',
      f'G92 E0.0 \n',
      f'G1 Z0.200 F10800.000 \n',
     
      f'\n',
      f'\n']

M1 = [f'G1 X{X0} Y{Y0} Z{Z0} \n',
      f'M204 S1000 \n',
      f'G1 X{X1} F{F1} E{E1} \n']

D2 = [f'\n',
      f'\n',
      
      f'G1 F10800.000 \n',
      f'G4 ; wait \n',
      f'M104 S0 ; turn off temperature \n',
      f'M140 S0 ; turn off heatbed \n',
      f'M107 ; turn off fan \n',
      f'G1 Z30.2 ; Move print head up \n',
      f'M73 P91 R0 \n',
      f'G1 X0 Y200 F3000 ; home X axis \n',
      f'M900 K0 ; reset LA \n',
      f'M84 ; disable motors \n',
      f'M73 P100 R0 \n']
  
# \n is placed to indicate EOL (End of Line) 
file1.writelines(D1)
file1.writelines(M1)
file1.writelines(D2)
file1.close() #to change file access modes 